In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [2]:
tf.keras.applications.efficientnet.EfficientNetB0()

In [3]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0

In [4]:
print("TF Version: ", tf.__version__)

TF Version:  2.6.0


In [5]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

In [6]:
if __name__ == "__main__":
    inputs = Input((256, 256, 3))
    x = conv_block(inputs, 32)
    print(x.shape)

(None, 256, 256, 32)


In [19]:
def build_efficient_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained encoder """
    encoder = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)

    encoder.summary()

In [20]:
if __name__ == "__main__":
  input_shape = (256, 256, 3)
  build_efficient_unet(input_shape)

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 256, 256, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 256, 256, 3)  7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 257, 257, 3)  0           normalization_2[0][0]            
_____________________________________________________________________________________

In [14]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

In [25]:
def build_efficient_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained encoder """
    encoder = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs)

    s1 = encoder.get_layer("input_10").output                        # 256
    s2 = encoder.get_layer("block2a_expand_activation").output      # 128
    s3 = encoder.get_layer("block3a_expand_activation").output      # 64
    s4 = encoder.get_layer("block4a_expand_activation").output      # 32

    """ Bottleneck """
    b1 = encoder.get_layer("block6a_expand_activation").output      # 16         

    """ Decoder """               
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    """ Output """
    outputs = Conv2D(1, 1, padding='same', activation='sigmoid')(d4)

    model = Model(inputs, outputs, name='EfficientNetB0_UNET')
    return model 

In [26]:
if __name__ == "__main__":
  input_shape = (256, 256, 3)
  build_efficient_unet(input_shape)
  model.summary

NameError: ignored